In [5]:
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.signal import medfilt

from numpy.fft import fft, fftshift, fftfreq
from matplotlib import pyplot as plt
from pathlib import Path
import ffmpeg, librosa

import helpers

# Driver

exerciseNoteCounts = [
    ["ex1WholeMod.mp4", 90, .3,  .4],
    ["ex1WholeModF.mp4", 90, .3, .4],
    ["ex2WholeMod.mp4", 49, .15, 0.15],
    ["ex3WholeMod.mp4", 145, .15, 0.15],
    ["ex3WholeModF.mp4", 145, .15, 0.15],
    ["ex4WholeMod.mp4", 102, .15, 0.15],
    ["ex4WholeModF.mp4", 102, .15, 0.15],
    ["ex5WholeMod.mp4", 133, .15, 0.15],
    ["ex5WholeModF.mp4", 133, .15, 0.15],
    ["ex6WholeMod.mp4", 118, .15, 0.15],
    ["ex6WholeModF.mp4", 118, .15, 0.15],
    ["ex7WholeMod.mp4", 86, .15, 0.15],
    ["ex8WholeMod.mp4", 112, .15, 0.15],
    ["ex8WholeModF.mp4", 112, .15, 0.15],
    ["ex9WholeMod.mp4", 121, .15, 0.15],
    ["ex10WholeMod.mp4", 102, .15, 0.15]
]

for exercise, note_count, spec_thresh, min_time_between in exerciseNoteCounts:
    ys, ts, sr = helpers.get_audio_data(exercise)

    ts_fr, fr_freq_amps = helpers.magnitudeSpectrogram(ys, ts, sr)
    ts_fr, spec_flux = helpers.computeSpectralFlux(ts_fr, fr_freq_amps, sr)

    plt.title(exercise)
    plt.xlabel("Time (s)"); plt.ylabel("Spectral Flux")
    plt.plot(ts_fr, spec_flux, c="orange")
    img_path = Path.cwd() / "spec_flux_graphs" / f"{exercise} Spectral Flux.png"
    plt.savefig(img_path)
    plt.close()
    
    f0_frame_size = 2048
    f0_hop_len = 512 
    f0, voiced, _ = librosa.pyin(y=ys, sr=sr, fmin=librosa.note_to_hz('E3'), fmax=librosa.note_to_hz('C6'), frame_length=f0_frame_size, hop_length=f0_hop_len)
    # f0 = gaussian_filter1d(f0, sigma=1)

    onsets = helpers.detect_onsets(spec_flux, ts_fr, threshold=spec_thresh, min_time_between=min_time_between, f0=f0, sr=sr, voiced=voiced, hop_len=f0_hop_len)

    print(f"For exercise {exercise}, Note count Actual:\t{note_count}, Note count Detected: \t{len(onsets)} | Accuracy: \t{min(note_count / len(onsets), len(onsets) / note_count)}")

For exercise ex1WholeMod.mp4, Note count Actual:	90, Note count Detected: 	47 | Accuracy: 	0.5222222222222223
For exercise ex1WholeModF.mp4, Note count Actual:	90, Note count Detected: 	38 | Accuracy: 	0.4222222222222222
For exercise ex2WholeMod.mp4, Note count Actual:	49, Note count Detected: 	147 | Accuracy: 	0.3333333333333333
For exercise ex3WholeMod.mp4, Note count Actual:	145, Note count Detected: 	232 | Accuracy: 	0.625
For exercise ex3WholeModF.mp4, Note count Actual:	145, Note count Detected: 	149 | Accuracy: 	0.9731543624161074
For exercise ex4WholeMod.mp4, Note count Actual:	102, Note count Detected: 	203 | Accuracy: 	0.5024630541871922
For exercise ex4WholeModF.mp4, Note count Actual:	102, Note count Detected: 	149 | Accuracy: 	0.6845637583892618
For exercise ex5WholeMod.mp4, Note count Actual:	133, Note count Detected: 	279 | Accuracy: 	0.4767025089605735
For exercise ex5WholeModF.mp4, Note count Actual:	133, Note count Detected: 	221 | Accuracy: 	0.6018099547511312
For ex